In [1]:
import pandas as pd

from ltlcross_wrapper import GoalModulizer, ResAnalyzer
from tools_goal_variants import tools
from tools import benchmark_names as names
from tools import goal_root

In [2]:
res_dir = "data.goal_variants"
!mkdir -p {res_dir}

## GOAL piterman complement options

In [3]:
def gather_cumulative(benchmarks, tools=None, transpose=True, **kwargs):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = pd.DataFrame(b.cumulative(tool_set=tools, highlight=False, **kwargs))
        tmp.columns = [name]
        data = data.append(tmp.transpose())
    if transpose:
        return data.transpose().style.apply(highlight_min, axis=0)
    else:
        return data.style.apply(highlight_min, axis=1)
    
def gather_mins(benchmarks, tools=None, transpose=True):
    data = pd.DataFrame()
    for (name,b) in benchmarks.items():
        tmp = b.min_counts(tool_set=tools, restrict_tools=True, )
        tmp.columns = pd.MultiIndex.from_tuples([(name, c) for c in tmp.columns])
        data = data.append(tmp.transpose(),sort=False).fillna(0)
    if transpose:
        return data.transpose().style.apply(highlight_max, axis=0)
    return data.style.apply(highlight_max, axis=1)

def highlight_min(s):    
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]

def highlight_max(s):    
    is_max = s == s.max()
    return ['background-color: lightgreen' if v else '' for v in is_max]

# Run ltlcross

## What is the most useful pipeline?

The option combinations and `-sp -ro` and `-eq -sp -ro` are bogus and thus we have to remove them from evaluation. We have selected the `-eq -sp` as the goal pipeline.

In [4]:
tool_set = [t for t in tools.keys() if t[-3:] == "yes"]
tool_set = [t for t in tool_set if t not in ["spro.yes","eqspro.yes"]]

In [5]:
benchmarks = {}
for name in names:
    b = ResAnalyzer(f"{res_dir}/GV.{name}.csv", tool_set=tool_set, cols=["states","time","acc","transitions","edges"])
    b.name = name
    benchmarks[name] = b

In [6]:
gather_cumulative(benchmarks)

,literature_det,literature_sd,literature_nd,random_det,random_sd,random_nd
tool,,,,,,
eq.yes,701,567,290,2493,5445,8815
eqro.yes,707,563,278,2490,5207,8527
eqsp.yes,697,331,271,2451,3333,7599
ht.yes,701,497,279,2493,4813,8360
hteq.yes,701,455,272,2493,4768,8311
hteqro.yes,701,455,272,2493,4768,8311
hteqrosp.yes,697,331,271,2451,3364,7623
htro.yes,701,497,279,2493,4813,8360
htsp.yes,697,333,276,2451,3379,7647


## Minimal automata
The follwing table shows for how many formulas each tool produces automaton that has the smallest number of states. The minimum ranges over the considered tools. The number in `min hits` shows how many times the same size as the smallest automaton was achieved. The number in `unique min hits` counts only cases where the given tool is the only tool with such a small automaton.

In [7]:
gather_mins(benchmarks)

In [8]:
b = benchmarks["random_nd"]

In [9]:
b.cross_compare(["eqsp.yes","hteqrosp.yes"])

,eqsp.yes,hteqrosp.yes,V
eqsp.yes,nan,20,20
hteqrosp.yes,13,nan,13
